# Feature Selection and LASSO 

In this notebook, we will use LASSO to select features, building on a pre-implemented solver for LASSO. 
** outline for this notebook**

* We will run LASSO with different L1 penalties.
* We will choose best L1 penalty using a validation set.
* We will choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, we will implement our own LASSO solver, using coordinate descent. 

## import library

In this section, we will import library for later use.

In [5]:
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from math import log, sqrt
import seaborn as sns
sns.set(color_codes=True)
import matplotlib.pyplot as plt

## Read data in

Dataset used in this notebook is from house sales in King County, the region where the city of Seattle, WA is located.

In [7]:
data = pd.read_csv("kc_house_data.csv")
colname_lst = list(df.columns.values)
coltype_lst =  [str, str, float, float, float, float, int, str, int, int, int, int, int, int, int, int, str, float, float, float, float]
col_type_dict = dict(zip(colname_lst, coltype_lst))
data.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900,3,1.00,1180,5650,1,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000,3,2.25,2570,7242,2,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000,2,1.00,770,10000,1,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000,4,3.00,1960,5000,1,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000,3,2.00,1680,8080,1,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


## create new features

In this notebook, we consider to transform some input features.

In [8]:
data['sqft_living_sqrt'] = data['sqft_living'].apply(sqrt)
data['sqft_lot_sqrt'] = data['sqft_lot'].apply(sqrt)
data['bedrooms_square'] = data['bedrooms']*data['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
data['floors'] = data['floors'].astype(float) 
data['floors_square'] = data['floors']*data['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [17]:
all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in GraphLab Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [18]:
model_all = graphlab.linear_regression.create(sales, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=1e10)

Linear regression:

--------------------------------------------------------

Number of examples          : 21613

Number of features          : 17

Number of unpacked features : 17

...

| 5         | 6        | 0.000002  | 1.327137     | 6869667.895833     | 385998.458623 |

| 6         | 7        | 0.000002  | 1.413999     | 6847177.773672     | 380824.455891 |

+-----------+----------+-----------+--------------+--------------------+---------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

Find what features had non-zero weight.

In [19]:
model_all.get('coefficients').print_rows(num_rows=18, num_columns=3)

+------------------+-------+---------------+-----+
|       name       | index |     value     | ... |
+------------------+-------+---------------+-----+
|   (intercept)    |  None |  274873.05595 | ... |
|     bedrooms     |  None |      0.0      | ... |
| bedrooms_square  |  None |      0.0      | ... |
|    bathrooms     |  None | 8468.53108691 | ... |
|   sqft_living    |  None | 24.4207209824 | ... |
| sqft_living_sqrt |  None | 350.060553386 | ... |
|     sqft_lot     |  None |      0.0      | ... |
|  sqft_lot_sqrt   |  None |      0.0      | ... |
|      floors      |  None |      0.0      | ... |
|  floors_square   |  None |      0.0      | ... |
|    waterfront    |  None |      0.0      | ... |
|       view       |  None |      0.0      | ... |
|    condition     |  None |      0.0      | ... |
|      grade       |  None | 842.068034898 | ... |
|    sqft_above    |  None | 20.0247224171 | ... |
|  sqft_basement   |  None |      0.0      | ... |
|     yr_built     |  None |   

Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [20]:
(training_and_validation, testing) = sales.random_split(.9,seed=1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [21]:
l1_penality = np.logspace(1, 7, num=13)
print l1_penality

[  1.00000000e+01   3.16227766e+01   1.00000000e+02   3.16227766e+02
   1.00000000e+03   3.16227766e+03   1.00000000e+04   3.16227766e+04
   1.00000000e+05   3.16227766e+05   1.00000000e+06   3.16227766e+06
   1.00000000e+07]


In [22]:
rss_validation =[]
rss_max = float('inf')
for penalty in l1_penality:
    model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, verbose = False,
                                              l2_penalty=0., l1_penalty=penalty)
    prediction = model.predict(validation)
    residules = prediction - validation['price']
    rss = (residules * residules).sum()
    rss_validation.append(rss)
    if rss < rss_max:
        rss_max = rss
        model_with_best_rss = model
print rss_validation
print 'best rss for validation set', rss_max

[625766285142459.9, 625766285362394.1, 625766286057885.0, 625766288257224.6, 625766295212186.8, 625766317206080.5, 625766386760658.1, 625766606749278.5, 625767302791634.1, 625769507643886.2, 625776517727024.0, 625799062845467.0, 625883719085425.2]
best rss for validation set 6.25766285142e+14


*** QUIZ QUESTIONS ***
1. What was the best value for the `l1_penalty`?
2. What is the RSS on TEST data of the model with the best `l1_penalty`?

In [23]:
residule_test = model_with_best_rss.predict(testing) - testing['price']
print (residule_test * residule_test).sum()

1.56983602382e+14


***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have?

In [24]:
model_with_best_rss.get('coefficients').print_rows(num_rows=18, num_columns=3)

+------------------+-------+------------------+-----+
|       name       | index |      value       | ... |
+------------------+-------+------------------+-----+
|   (intercept)    |  None |  18993.4272128   | ... |
|     bedrooms     |  None |  7936.96767903   | ... |
| bedrooms_square  |  None |  936.993368193   | ... |
|    bathrooms     |  None |  25409.5889341   | ... |
|   sqft_living    |  None |  39.1151363797   | ... |
| sqft_living_sqrt |  None |  1124.65021281   | ... |
|     sqft_lot     |  None | 0.00348361822299 | ... |
|  sqft_lot_sqrt   |  None |  148.258391011   | ... |
|      floors      |  None |   21204.335467   | ... |
|  floors_square   |  None |  12915.5243361   | ... |
|    waterfront    |  None |  601905.594545   | ... |
|       view       |  None |  93312.8573119   | ... |
|    condition     |  None |  6609.03571245   | ... |
|      grade       |  None |  6206.93999188   | ... |
|    sqft_above    |  None |  43.2870534193   | ... |
|  sqft_basement   |  None |

# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [25]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [26]:
l1_penalty_values = np.logspace(8, 10, num=20)

Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model['coefficients']['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [27]:
nzz_l1_penalty_values = []
for penalty in l1_penalty_values:
    model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, verbose = False,
                                              l2_penalty=0., l1_penalty=penalty)
    nzz_l1_penalty_values.append(model['coefficients']['value'].nnz())
print nzz_l1_penalty_values

[18, 18, 18, 18, 17, 17, 17, 17, 17, 16, 15, 15, 13, 12, 10, 6, 5, 3, 1, 1]


Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzero` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzero` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [34]:
i = 0
while (nzz_l1_penalty_values[i] > max_nonzeros):
    i += 1
l1_penalty_min = l1_penalty_values[i - 1]
print 'largest l1 penalty %s with nzz more than max-non-zero index %s' % (l1_penalty_min, i-1) 
l1_penalty_max = l1_penalty_values[i]
print 'smallest l1 penalty %s with nzz less than max-non-zero index %s' % (l1_penalty_max, i) 

largest l1 penalty 3577864204.13 with nzz more than max-non-zero index 14
smallest l1 penalty 3620829401.45 with nzz less than max-non-zero index 15


***QUIZ QUESTIONS***

What values did you find for `l1_penalty_min` and`l1_penalty_max`? 

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [38]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)
print l1_penalty_values

[  3.57786420e+09   3.58012553e+09   3.58238686e+09   3.58464818e+09
   3.58690951e+09   3.58917084e+09   3.59143216e+09   3.59369349e+09
   3.59595481e+09   3.59821614e+09   3.60047747e+09   3.60273879e+09
   3.60500012e+09   3.60726144e+09   3.60952277e+09   3.61178410e+09
   3.61404542e+09   3.61630675e+09   3.61856808e+09   3.62082940e+09]


* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzero`.

In [39]:
rss_validation2 =[]
rss_max2 = float('inf')

for penalty in l1_penalty_values:
    model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, verbose = False,
                                              l2_penalty=0., l1_penalty=penalty)
    if (model['coefficients']['value'].nnz() == max_nonzeros):
        prediction = model.predict(validation)
        residules = prediction - validation['price']
        rss = (residules * residules).sum()
        rss_validation.append(rss)
        if rss < rss_max2:
            l1_penalty_with_lowest_rss_max_nonzero = penalty
            rss_max2 = rss
            model_with_best_rss2 = model
print rss_validation2
print 'best rss for validation set in narrow range of l1 penalty', rss_max2
print 'l1_penalty with lowerst rss on validation and max non zero', l1_penalty_with_lowest_rss_max_nonzero
model_with_best_rss2.get('coefficients').print_rows(num_rows=18, num_columns=3)

[]
best rss for validation set in narrow range of l1 penalty 1.06079953176e+15
l1_penalty with lowerst rss on validation and max non zero 3577864204.13
+------------------+-------+---------------+-----+
|       name       | index |     value     | ... |
+------------------+-------+---------------+-----+
|   (intercept)    |  None | 229077.875448 | ... |
|     bedrooms     |  None | 149.417508491 | ... |
| bedrooms_square  |  None |      0.0      | ... |
|    bathrooms     |  None | 15146.7392756 | ... |
|   sqft_living    |  None | 31.7586377842 | ... |
| sqft_living_sqrt |  None |  655.07824878 | ... |
|     sqft_lot     |  None |      0.0      | ... |
|  sqft_lot_sqrt   |  None |      0.0      | ... |
|      floors      |  None |      0.0      | ... |
|  floors_square   |  None |      0.0      | ... |
|    waterfront    |  None |      0.0      | ... |
|       view       |  None |      0.0      | ... |
|    condition     |  None |      0.0      | ... |
|      grade       |  None | 267

***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?
2. What features in this model have non-zero coefficients?